In [189]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

## Extracting the array of migration relevant CAMEO events

In [190]:
CAMEOcodes_handle = pd.read_csv("../data/CAMEO_table_clean.csv")

##### I have to drop the first rows and take then only the inluded yes variables

In [191]:
CAMEOs = CAMEOcodes_handle.copy()

In [192]:
selectedCodes = CAMEOs.iloc[:, 0].str.extract(r"^(.*):")

In [193]:
selected_np = selectedCodes.iloc[:, 0].to_numpy()

In [194]:
selected = selected_np.tolist()

In [195]:
selected = np.char.replace(selected, "Social", "6")

In [196]:
CAMEO_selection = np.empty_like(selected)
for i, item in np.ndenumerate(selected):
    CAMEO_selection[i] = re.sub(r"\D", "", item)

In [197]:
CAMEO_selection.tolist()

['0232',
 '0234',
 '0254',
 '0255',
 '0256',
 '026',
 '027',
 '028',
 '0354',
 '0355',
 '0356',
 '036',
 '037',
 '038',
 '081',
 '0871',
 '0872',
 '0873',
 '0874',
 '093',
 '094',
 '1012',
 '1014',
 '1032',
 '1034',
 '1054',
 '1055',
 '1056',
 '106',
 '107',
 '108',
 '1123',
 '1124',
 '1125',
 '1244',
 '1245',
 '1246',
 '125',
 '126',
 '127',
 '138',
 '1382',
 '1383',
 '1384',
 '1385',
 '139',
 '150',
 '152',
 '154',
 '155',
 '164',
 '165',
 '166',
 '1661',
 '1662',
 '1663',
 '170',
 '176',
 '180',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '1951',
 '1952',
 '196',
 '200',
 '204',
 '2041',
 '2042',
 '1033',
 '023',
 '0231',
 '1011',
 '103',
 '1031',
 '024',
 '0241',
 '0244',
 '034',
 '0341',
 '0342',
 '0344',
 '035',
 '083',
 '0831',
 '0832',
 '0834',
 '091',
 '104',
 '1041',
 '1042',
 '1044',
 '105',
 '1121',
 '123',
 '1231',
 '1232',
 '1234',
 '124',
 '1241',
 '128',
 '0243',
 '0251',
 '0253',
 '0343',
 '0351',
 '0353',
 '075',
 '0811',
 '0812',
 '0813',
 '0814',
 '0833',
 '092

In [198]:
CAMEOs["code"] = pd.Series(data=CAMEO_selection)
CAMEOs

,GDELT code and topic,Included,Sign,Category\nlabel,Strength,code
0,0232: Appeal for military aid,Yes,+,Conflict,1,0232
1,0234: Appeal for military protection or peacek...,Yes,+,Conflict,1,0234
2,"0254: Appeal for easing of economic sanctions,...",Yes,+,Economic,1,0254
3,0255: Appeal for target to allow international...,Yes,+,Conflict,1,0255
4,025: Social: Appeal for de-escalation of milit...,Yes,+,Conflict,1,0256
...,...,...,...,...,...,...
232,"1Social2: Reduce or stop material aid, not spe...",Yes,+,Conflict,2,162
233,1Social21: Reduce or stop economic assistance,Yes,+,Economic,2,1621
234,1Social22: Reduce or stop military assistance,Yes,+,Conflict,2,1622
235,1Social23: Reduce or stop humanitarian assistance,Yes,+,Conflict,2,1623


##### Checking the relevance of the general root CAMEO tags

In [199]:
# In the CAMEO code selection table of the paper, they left unspecified general CAMEO class codes:
CAMEOs[CAMEOs["code"].str.len() == 2]

,GDELT code and topic,Included,Sign,Category\nlabel,Strength,code
194,07: PROVIDE AID,Yes,-,Conflict,1,07
205,13: THREATEN,Yes,+,Conflict,1,13
206,1Social: REDUCE RELATIONS,Yes,+,Conflict,1,16
230,14: PROTEST,Yes,+,Social,1,14
231,15: EXHIBIT FORCE POSTURE,Yes,+,Conflict,1,15


##### Deciding, for simplicity purposes, drop the general CAMEO codes

In [200]:
relevant_CAMEOS = CAMEOs.drop(CAMEOs[CAMEOs["code"].str.len() == 2].index)
relevant_CAMEOS = CAMEOs.drop("Included", axis=1)
relevant_CAMEOS

,GDELT code and topic,Sign,Category\nlabel,Strength,code
0,0232: Appeal for military aid,+,Conflict,1,0232
1,0234: Appeal for military protection or peacek...,+,Conflict,1,0234
2,"0254: Appeal for easing of economic sanctions,...",+,Economic,1,0254
3,0255: Appeal for target to allow international...,+,Conflict,1,0255
4,025: Social: Appeal for de-escalation of milit...,+,Conflict,1,0256
...,...,...,...,...,...
232,"1Social2: Reduce or stop material aid, not spe...",+,Conflict,2,162
233,1Social21: Reduce or stop economic assistance,+,Economic,2,1621
234,1Social22: Reduce or stop military assistance,+,Conflict,2,1622
235,1Social23: Reduce or stop humanitarian assistance,+,Conflict,2,1623


In [201]:
relevant_CAMEOS = relevant_CAMEOS.rename(
    columns={
        "GDELT code and topic": "description",
        "Sign": "sign",
        "Category\nlabel": "label",
        "Strength": "weight",
    }
)

In [202]:
relevant_CAMEOS["weight"] = relevant_CAMEOS["sign"] + relevant_CAMEOS["weight"].astype(
    "str"
)

In [203]:
relevant_CAMEOS["weight"] = relevant_CAMEOS["weight"].astype("int")

In [204]:
relevant_CAMEOS = relevant_CAMEOS.drop("sign", axis=1)

In [205]:
relevant_CAMEOS = relevant_CAMEOS.loc[:, ::-1]

In [206]:
relevant_CAMEOS["label"] = (
    relevant_CAMEOS["label"].str.lower().replace("governanc\ne", "governance")
)

In [207]:
relevant_CAMEOS.to_csv("../data/CAMEO_weights.csv")

## Preprocessing GDELT 1.0

#### Adding the column names first.

In [ ]:
header = "GLOBALEVENTID   SQLDATE MonthYear   Year    FractionDate	Actor1Code	Actor1Name	Actor1CountryCode	Actor1KnownGroupCode	Actor1EthnicCode	Actor1Religion1Code	Actor1Religion2Code	Actor1Type1Code	Actor1Type2Code	Actor1Type3Code	Actor2Code	Actor2Name	Actor2CountryCode	Actor2KnownGroupCode	Actor2EthnicCode	Actor2Religion1Code	Actor2Religion2Code	Actor2Type1Code	Actor2Type2Code	Actor2Type3Code	IsRootEvent	EventCode	EventBaseCode	EventRootCode	QuadClass	GoldsteinScale	NumMentions	NumSources	NumArticles	AvgTone	Actor1Geo_Type	Actor1Geo_FullName	Actor1Geo_CountryCode	Actor1Geo_ADM1Code	Actor1Geo_Lat	Actor1Geo_Long	Actor1Geo_FeatureID	Actor2Geo_Type	Actor2Geo_FullName	Actor2Geo_CountryCode	Actor2Geo_ADM1Code	Actor2Geo_Lat	Actor2Geo_Long	Actor2Geo_FeatureID	ActionGeo_Type	ActionGeo_FullName	ActionGeo_CountryCode	ActionGeo_ADM1Code	ActionGeo_Lat	ActionGeo_Long	ActionGeo_FeatureID	DATEADDED	SOURCEURL"
header = header.split()
header

['GLOBALEVENTID',
 'SQLDATE',
 'MonthYear',
 'Year',
 'FractionDate',
 'Actor1Code',
 'Actor1Name',
 'Actor1CountryCode',
 'Actor1KnownGroupCode',
 'Actor1EthnicCode',
 'Actor1Religion1Code',
 'Actor1Religion2Code',
 'Actor1Type1Code',
 'Actor1Type2Code',
 'Actor1Type3Code',
 'Actor2Code',
 'Actor2Name',
 'Actor2CountryCode',
 'Actor2KnownGroupCode',
 'Actor2EthnicCode',
 'Actor2Religion1Code',
 'Actor2Religion2Code',
 'Actor2Type1Code',
 'Actor2Type2Code',
 'Actor2Type3Code',
 'IsRootEvent',
 'EventCode',
 'EventBaseCode',
 'EventRootCode',
 'QuadClass',
 'GoldsteinScale',
 'NumMentions',
 'NumSources',
 'NumArticles',
 'AvgTone',
 'Actor1Geo_Type',
 'Actor1Geo_FullName',
 'Actor1Geo_CountryCode',
 'Actor1Geo_ADM1Code',
 'Actor1Geo_Lat',
 'Actor1Geo_Long',
 'Actor1Geo_FeatureID',
 'Actor2Geo_Type',
 'Actor2Geo_FullName',
 'Actor2Geo_CountryCode',
 'Actor2Geo_ADM1Code',
 'Actor2Geo_Lat',
 'Actor2Geo_Long',
 'Actor2Geo_FeatureID',
 'ActionGeo_Type',
 'ActionGeo_FullName',
 'ActionGeo_Co

In [ ]:
handle = pd.read_table(
    "../data/gdelt_data/extracted/20230511.export.CSV",
    names=header,
    dtype={"EventCode": str},
)
handle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174898 entries, 0 to 174897
Data columns (total 58 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   GLOBALEVENTID          174898 non-null  int64  
 1   SQLDATE                174898 non-null  int64  
 2   MonthYear              174898 non-null  int64  
 3   Year                   174898 non-null  int64  
 4   FractionDate           174898 non-null  float64
 5   Actor1Code             158608 non-null  object 
 6   Actor1Name             158608 non-null  object 
 7   Actor1CountryCode      99651 non-null   object 
 8   Actor1KnownGroupCode   2400 non-null    object 
 9   Actor1EthnicCode       881 non-null     object 
 10  Actor1Religion1Code    1970 non-null    object 
 11  Actor1Religion2Code    456 non-null     object 
 12  Actor1Type1Code        76221 non-null   object 
 13  Actor1Type2Code        4993 non-null    object 
 14  Actor1Type3Code        104 non-null 

In [ ]:
handle.to_csv("../data/gdelt_data/gdelt_example.csv")

In [ ]:
handle["GLOBALEVENTID"].value_counts().value_counts()

1    174898
Name: GLOBALEVENTID, dtype: int64

#### Defining feature extraction functions for a gdelt daily event table

In [170]:
CAMEO_weights = pd.read_csv(
    "../data/CAMEO_weights.csv", index_col=0, dtype={"code": str}
)
CAMEO_weights = CAMEO_weights.set_index("code")
CAMEO_weights

In [118]:
handle = pd.read_csv(
    "../data/gdelt_data/gdelt_example.csv", index_col=0, dtype={"EventCode": str}
)

In [174]:
eventdf = handle.copy()
eventdf

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1101511638,20220511,202205,2022,2022.3589,NaN,NaN,NaN,NaN,NaN,...,1660757,3,"Hollywood, California, United States",US,USCA,34.0983,-118.32700,1660757,20230511,https://www.ktvq.com/news/local-news/labor-of-...
1,1101511639,20220511,202205,2022,2022.3589,NaN,NaN,NaN,NaN,NaN,...,1492837,3,"Danville, Virginia, United States",US,USVA,36.5860,-79.39500,1492837,20230511,https://godanriver.com/news/local/govt-and-pol...
2,1101511640,20220511,202205,2022,2022.3589,NaN,NaN,NaN,NaN,NaN,...,1492837,2,"Virginia, United States",US,USVA,37.7680,-78.20570,VA,20230511,https://godanriver.com/news/local/govt-and-pol...
3,1101511641,20220511,202205,2022,2022.3589,NaN,NaN,NaN,NaN,NaN,...,1660757,3,"Hollywood, California, United States",US,USCA,34.0983,-118.32700,1660757,20230511,https://www.ktvq.com/news/local-news/labor-of-...
4,1101511642,20220511,202205,2022,2022.3589,BUS,COMPANY,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,20230511,https://finance.yahoo.com/news/zheneng-jinjian...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174893,1101765385,20230511,202305,2023,2023.3589,ZWE,ZIMBABWE,ZWE,NaN,NaN,...,-2312291,4,"Harare, Mashonaland East, Zimbabwe",ZI,ZI04,-17.8178,31.04470,-2312291,20230511,https://www.herald.co.zw/culture-month-launched/
174894,1101765386,20230511,202305,2023,2023.3589,ZWE,ZIMBABWE,ZWE,NaN,NaN,...,-2312291,4,"Harare, Mashonaland East, Zimbabwe",ZI,ZI04,-17.8178,31.04470,-2312291,20230511,https://www.herald.co.zw/culture-month-launched/
174895,1101765387,20230511,202305,2023,2023.3589,chm,MARI,NaN,NaN,chm,...,RP,1,Philippines,RP,RP,13.0000,122.00000,RP,20230511,https://www.hellenicshippingnews.com/a-friendl...
174896,1101765388,20230511,202305,2023,2023.3589,chm,MARI,NaN,NaN,chm,...,-2597039,4,"Glasgow, Glasgow City, United Kingdom",UK,UKV2,55.8333,-4.25000,-2597039,20230511,https://www.hellenicshippingnews.com/a-friendl...


In [173]:
def eventdf_FE(eventdf, CAMEOtags):
    # Extract relevant features of the big GDELT day table
    df = eventdf[
        [
            "SQLDATE",
            "IsRootEvent",
            "EventCode"
            # ,'QuadClass'           Excluded as we build the event indices proposed by the paper
            # ,'GoldsteinScale'
            ,
            "ActionGeo_CountryCode"
            # ,'SOURCEURL'           As we are going to agregate all these for a month, it is not relevant anymore to track it back
        ]
    ].copy()
    # Extract only root events as done in the paper
    df = df[df["IsRootEvent"] == 1]
    df = df.drop("IsRootEvent", axis=1)
    # Extract only the relevant events
    df = df[df["EventCode"].isin(CAMEOtags)]
    # drop NaNs, as they are mostly in the CountryCode
    df = df.dropna()
    df = df.reset_index(drop=True)
    df = df.rename(
        columns={
            "SQLDATE": "date",
            "EventCode": "code",
            "ActionGeo_CountryCode": "country",
        }
    )
    return df

In [176]:
def get_category_weights(eventdf, CAMEO_weights):
    # Filter the giant day GDELT event data to its relevant columns
    df = eventdf_FE(eventdf, CAMEO_weights.index.tolist())
    # Aggregate them by country and event-type-code
    # and get the quantity of the single event-types for in each country
    df = df.groupby(["country", "code"]).size()
    df = pd.DataFrame(df).reset_index().rename(columns={0: "count"})
    # Now we are associating each event-type to their corresponding
    # weights and labels describe in the CAMEO_weights reference table
    df["category"] = pd.NA
    for i, row in df.loc[:, ["code"]].iterrows():
        weight = CAMEO_weights.loc[row["code"]]["weight"]
        count = df["count"].iloc[i]
        df["count"].iloc[i] = count * weight / 3
        df["category"].iloc[i] = CAMEO_weights.loc[row["code"]]["label"]
    # Done this, we don't need the code column anymore
    # so we drop it, an agreggate by country and category
    df = df.drop("code", axis=1).groupby(["country", "category"]).sum()
    return df

In [178]:
day_weights = get_category_weights(eventdf, CAMEO_weights)
day_weights

/tmp/ipykernel_7794/3861380352.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count'].iloc[i] = count*weight/3
/tmp/ipykernel_7794/3861380352.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'].iloc[i] = CAMEO_weights.loc[row['code']]['label']


In [161]:
ec2 = day_weights.copy()
ec2["count"] + day_weights["count"]

country  category 
AA       Conflict      7.333333
         Political    47.333333
AC       Conflict      0.000000
AE       Conflict    -30.000000
         Economic    -41.333333
                        ...    
ZA       Social        0.666667
ZI       Conflict     16.000000
         Economic    -26.666667
         Political    41.333333
         Social        4.666667
Name: count, Length: 689, dtype: float64

##### Next steps: to extract the push factors, aggregate them by week and or month, following the event indeces methodology in the paper

In [67]:
eventdf

,SQLDATE,EventCode,ActionGeo_CountryCode
0,20220511,180,US
1,20220511,180,US
2,20230411,190,PK
3,20230411,190,PK
4,20230411,190,PK
...,...,...,...
33894,20230511,070,CE
33895,20230511,070,CE
33896,20230511,070,VM
33897,20230511,0841,VM


In [ ]:
def Extract_PFI():
    pass

In [39]:
eventdf.dropna().isna().sum()

SQLDATE                  0
EventCode                0
QuadClass                0
GoldsteinScale           0
ActionGeo_CountryCode    0
SOURCEURL                0
dtype: int64